# Агрегирование и группировка 

## Данные о планетах

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
planets = sns.load_dataset('planets')  # информация об открытых астрономами планетах, вращающихся вокруг других звезд - экзопланет
planets.shape

(1035, 6)

In [2]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


Этот набор данных содержит определенную информацию о более чем 1000 экзопланет, открытых до 2014 года.

## Простое агрегирование в библиотеке Pandas 

Как и в случае одномерных массивов библиотеки NumPy,
для объектов Series библиотеки Pandas агрегирующие функции возвращают скалярное значение:

In [3]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [4]:
ser.sum() # сутта

2.811925491708157

In [5]:
ser.mean() # среднее

0.5623850983416314

В случае объекта DataFrame по умолчанию агрегирующие функции возвращают
сводные показатели по каждому столбцу:

In [6]:
df = pd.DataFrame({'A': rng.rand(5), 'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [7]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

Можно вместо этого агрегировать и по строкам, задав аргумент axis:

In [8]:
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

Объекты Series и DataFrame библиотеки Pandas содержат методы, соответствующие распространенным агрегирующим функциям. В них есть
удобный метод describe(), вычисляющий сразу несколько самых распространенных сводных показателей для каждого столбца и возвращающий результат.
Опробуем его на наборе данных «Планеты», пока удалив строки с отсутствующими
значениями:

In [9]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


Эта возможность очень удобна для первоначального знакомства с общими характеристиками нашего набора данных. Например, мы видим в столбце year, что,
хотя первая экзопланета была открыта еще в 1989 году, половина всех известных
экзопланет открыта не ранее 2010 года. В значительной степени мы обязаны этим
миссии «Кеплер», представляющей собой космический телескоп, специально разработанный для поиска затмений от планет, вращающих вокруг других звезд.

Для более глубокого исследования данных простых сводных показателей часто
недостаточно. Следующий уровень обобщения данных — операция groupby, позволяющая быстро и эффективно вычислять сводные показатели по подмножествам
данных.

## GroupBy: разбиение, применение, объединение

Простые агрегирующие функции дают возможность «прочувствовать» набор данных, но зачастую бывает нужно выполнить условное агрегирование по какой-либо
метке или индексу. Это действие реализовано в так называемой операции GroupBy.
Название group by («сгруппировать по») ведет начало от одноименной команды
в языке SQL баз данных, но, возможно, будет понятнее говорить о ней в терминах,
придуманных Хэдли Викхэмом, более известным своими библиотеками для языка R: разбиение, применение и объединение.

In [10]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'], 'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


Простейшую операцию «разбить, применить, объединить» можно реализовать
с помощью метода groupby() объекта DataFrame, передав в него имя желаемого
ключевого столбца:

In [11]:
df.groupby('key')

Возвращаемое — не набор объектов DataFrame, а объект
DataFrameGroupBy. Этот объект особенный, его можно рассматривать как специальное представление объекта DataFrame, готовое к группировке, но не выполняющее
никаких фактических вычислений до этапа применения агрегирования. Подобный
метод «отложенного вычисления» означает возможность очень эффективной
реализации распространенных агрегирующих функций, причем практически прозрачным для пользователя образом.
Для получения результата можно вызвать один из агрегирующих методов этого
объекта DataFrameGroupBy, что приведет к выполнению соответствующих шагов
применения/объединения:

In [12]:
df.groupby('key').sum() # сумма

,data
key,
A,3
B,5
C,7


In [13]:
df.groupby('key').prod() # умножение

,data
key,
A,0
B,4
C,10


In [14]:
df.groupby('key').mean()  # среднее

,data
key,
A,1.5
B,2.5
C,3.5


Здесь можно
использовать практически любую распространенную агрегирующую функцию
библиотек Pandas или NumPy, равно как и практически любую корректную операцию объекта DataFrame.

### Объект GroupBy 

#### Индексация по столбцам 

In [15]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [16]:
planets.groupby('method')

In [17]:
planets.groupby('method')['orbital_period']

Здесь мы выбрали конкретную группу Series из исходной группы DataFrame, сославшись на соответствующее имя столбца. Как и в случае с объектом GroupBy,
никаких вычислений не происходит до вызова для этого объекта какого-нибудь
агрегирующего метода:

In [18]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

Результат дает нам общее представление о масштабе чувствительности каждого из
методов к периодам обращения (в днях).

#### Цикл по группам 

Объект GroupBy поддерживает непосредственное выполнение циклов по группам с возвратом каждой группы в виде объекта Series или DataFrame:

In [19]:
for (method, group) in planets.groupby('method'):
    print('{0:30s} shape={1}'.format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


Это может пригодиться для выполнения некоторых вещей вручную, хотя обычно
быстрее воспользоваться встроенной функциональностью apply.

#### Методы диспечеризации

Благодаря определенной магии классов языка Python все
методы, не реализованные явным образом объектом GroupBy, будут передаваться далее
и выполняться для групп, вне зависимости от того, являются ли они объектами Series
или DataFrame. Например, можно использовать метод describe() объекта DataFrame
для вычисления набора сводных показателей, описывающих каждую группу в данных:

In [20]:
pd.set_option('display.max_rows', None) # показать все строки в выводе
planets.groupby('method')['year'].describe().unstack()

       method                       
count  Astrometry                          2.000000
       Eclipse Timing Variations           9.000000
       Imaging                            38.000000
       Microlensing                       23.000000
       Orbital Brightness Modulation       3.000000
       Pulsar Timing                       5.000000
       Pulsation Timing Variations         1.000000
       Radial Velocity                   553.000000
       Transit                           397.000000
       Transit Timing Variations           4.000000
mean   Astrometry                       2011.500000
       Eclipse Timing Variations        2010.000000
       Imaging                          2009.131579
       Microlensing                     2009.782609
       Orbital Brightness Modulation    2011.666667
       Pulsar Timing                    1998.400000
       Pulsation Timing Variations      2007.000000
       Radial Velocity                  2007.518987
       Transit             

Эта таблица позволяет получить лучшее представление о наших данных. Например, большинство планет было открыто методом измерения лучевой скорости
(radial velocity method) и транзитным методом (transit method), хотя последний
стал распространенным благодаря новым более точным телескопам только в последнее десятилетие. Похоже, что новейшими методами являются метод вариации
времени транзитов (transit timing variation method) и метод модуляции орбитальной яркости (orbital brightness modulation method), которые до 2011 года не использовались для открытия новых планет.

Это всего лишь один пример полезности методов диспетчеризации. Обратите
внимание, что они применяются к каждой отдельной группе, после чего результаты
объединяются в объект GroupBy и возвращаются. Можно использовать для соответствующего объекта GroupBy любой допустимый метод объектов Series/DataFrame,
что позволяет выполнять многие весьма гибкие и мощные операции!

### Агрегирование, фильтрация, преобразование, применение

Предыдущее обсуждение касалось агрегирования применительно к операции объединения, но доступны и другие возможности. В частности, у объектов GroupBy
имеются методы aggregate(), filter(), transform() и apply(), эффективно выполняющие множество полезных операций до объединения сгруппированных данных.

In [21]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                  columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


#### Агрегирование 

Метод aggregate() может принимать на входе строку, функцию
или список и вычислять все сводные показатели сразу. Вот пример, включающий
все вышеупомянутое:

In [37]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

Еще один удобный паттерн — передача в него словаря, связывающего имена столбцов с операциями, которые должны быть применены к этим столбцам:

In [23]:
df.groupby('key').aggregate({'data1': 'min', 'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


#### Фильтрация

Операция фильтрации дает возможность опускать данные в зависимости от свойств группы. Например, нам может понадобиться оставить в результате
все группы, в которых стандартное отклонение превышает какое-либо критическое
значение:

In [24]:
def filter_func(x):
    return x['data2'].std() > 4

In [25]:
print(df)
print()
print(df.groupby('key').std())
print()
print(df.groupby('key').filter(filter_func))

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9


Функция filter() возвращает булево значение, определяющее, прошла ли группа
фильтрацию. В данном случае, поскольку стандартное отклонение группы A не превышает 4, она удаляется из итогового результата.

#### Преобразование

В то время как агрегирующая функция должна возвращать
сокращенную версию данных, преобразование может вернуть версию полного набора данных, преобразованную ради дальнейшей их перекомпоновки. При подобном преобразовании форма выходных данных совпадает с формой входных.
Распространенный пример — центрирование данных путем вычитания среднего
значения по группам:

In [26]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


In [27]:
df.groupby('key').transform(lambda x: x ** 3) # возвести каждое значение в 3 степень

,data1,data2
0,0,125
1,1,0
2,8,27
3,27,27
4,64,343
5,125,729


#### Метод apply() 

Метод apply() позволяет применять произвольную функцию к результатам группировки. В качестве параметра эта функция должна получать объект DataFrame, а возвращать или объект библиотеки Pandas (например, DataFrame,
Series), или скалярное значение, в зависимости от возвращаемого значения будет
вызвана соответствующая операция объединения.

Например, функция apply(), нормирующая первый столбец на сумму значений
второго:

In [28]:
def norm_by_data2(x):
    # x - объект DataFrame сгруппированных значений    
    x['data1'] /= x['data2'].sum()
    return x

In [29]:
print(df)
print()
print(df.groupby('key').apply(norm_by_data2))

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

  key     data1  data2
0   A  0.000000      5
1   B  0.142857      0
2   C  0.166667      3
3   A  0.375000      3
4   B  0.571429      7
5   C  0.416667      9


Функция apply() в GroupBy достаточно гибка. Единственное требование, чтобы она
принимала на входе объект DataFrame и возвращала объект библиотеки Pandas или
скалярное значение; что вы делаете внутри, остается на ваше усмотрение!

### Задание ключа разбиения

В представленных ранее простых примерах мы разбивали объект DataFrame по
одному столбцу. Это лишь один из многих вариантов задания принципа формирования групп, и мы сейчас рассмотрим некоторые другие возможности.

#### Список, массив, объект Series и индекс как ключи группировки

Ключ может
быть любым рядом или списком такой же длины, как и у объекта DataFrame. Например:

In [30]:
L = [0, 1, 0, 1, 2, 0]
print(df)
print()
print(df.groupby(L).sum())

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

   data1  data2
0      7     17
1      4      3
2      4      7


Разумеется, это значит, что есть еще один, несколько более длинный способ выполнить вышеприведенную операцию df.groupby('key'):

In [31]:
print(df)
print()
print(df.groupby(df['key']).sum())

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

     data1  data2
key              
A        3      8
B        5      7
C        7     12


#### Словарь или объект Series, связывающий индекс и группу

Еще один метод:
указать словарь, задающий соответствие значений индекса и ключей группировки:

In [32]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
print(df2); print(); print(df2.groupby(mapping).sum())

     data1  data2
key              
A        0      5
B        1      0
C        2      3
A        3      3
B        4      7
C        5      9

           data1  data2
consonant     12     19
vowel          3      8


#### Любая функция языка Python

Аналогично заданию соответствия можно передать функции groupby любую функцию, принимающую на входе значение индекса
и возвращающую группу:

In [33]:
print(df2)
print()
print(df2.groupby(str.lower).mean())

     data1  data2
key              
A        0      5
B        1      0
C        2      3
A        3      3
B        4      7
C        5      9

   data1  data2
a    1.5    4.0
b    2.5    3.5
c    3.5    6.0


#### Список допустимых ключей

Можно комбинировать любые из предыдущих вариантов ключей для группировки по мультииндексу:

In [34]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
a,vowel,1.5,4.0
b,consonant,2.5,3.5
c,consonant,3.5,6.0


### Пример группировки

В качестве примера соберем все это вместе в нескольких строках кода на языке
Python и подсчитаем количество открытых планет по методу открытия и десятилетию:

In [38]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


In [39]:
planets

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300000,7.10000,77.40,2006
1,Radial Velocity,1,874.774000,2.21000,56.95,2008
2,Radial Velocity,1,763.000000,2.60000,19.84,2011
3,Radial Velocity,1,326.030000,19.40000,110.62,2007
4,Radial Velocity,1,516.220000,10.50000,119.47,2009
5,Radial Velocity,1,185.840000,4.80000,76.39,2008
6,Radial Velocity,1,1773.400000,4.64000,18.15,2002
7,Radial Velocity,1,798.500000,NaN,21.41,1996
8,Radial Velocity,1,993.300000,10.30000,73.10,2008
9,Radial Velocity,2,452.800000,1.99000,74.79,2010
